### Object Detection

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import pandas as pd
import base64
import io
import math 
from math import sqrt
import argparse
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Suppress Matplotlib warnings
%matplotlib inline             

### Paths 

In [18]:
# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = 'C:\\Users\\DELL\\Downloads\\shortshirt.jpg'

# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = 'E:\\Graduation Project\\IMPLEMENTATION\\tensorflow\\workspace\\training_demo\\exported-models\\my_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = 'E:\\Graduation Project\\IMPLEMENTATION\\tensorflow\\workspace\\training_demo\\annotations\\label_map.pbtxt'

### Load The Model

In [19]:
# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "\\saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

print('Running inference for {}... '.format(IMAGE_PATHS), end='')


Loading model...Done! Took 14.415225505828857 seconds
Running inference for C:\Users\DELL\Downloads\shortshirt.jpg... 

### Visualize The Output

In [ ]:
image = cv2.imread(IMAGE_PATHS)
pilImg = Image.open(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections


# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)


image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print('Done')

# DISPLAYS OUTPUT IMAGE
cv2.imshow('final image',image_with_detections)
#image_with_detections.show()

#waits for user to press any key 
#(this is necessary to avoid Python kernel form crashing)
cv2.waitKey(0) 
  
#closing all open windows 
cv2.destroyAllWindows() 
# CLOSES WINDOW ONCE KEY IS PRESSED

### Image Similarity

### Loading Pre-Trained Feature Extraction Model

In [21]:
global embed
saved_model_path = 'E:\Graduation Project\Datasets\PRE TRAINED MOBILE NET'
embed = hub.KerasLayer(saved_model_path)

### Converting Images To Feature Vectors (Embeddings)

In [22]:
class TensorVector(object):

    def __init__(self, FileName=None):
        self.FileName = FileName

    def process(self):

        img = tf.io.read_file(self.FileName)
        img = tf.io.decode_jpeg(img, channels=3)
        img = tf.image.resize_with_pad(img, 224, 224)
        img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis, ...]
        features = embed(img)
        feature_set = np.squeeze(features)
        return list(feature_set)

### class to find the similarity between two feature vectors 

In [23]:
class Similarity:
  def __init__(self, vector1, vector2):
    self.vector1 = vector1
    self.vector2 = vector2
  def cosineSimilarity(self):
      sum = 0
      suma1 = 0
      sumb1 = 0
      for i,j in zip(self.vector1, self.vector2):
          suma1 += i * i
          sumb1 += j*j
          sum += i*j
      cosine_sim = sum / ((sqrt(suma1))*(sqrt(sumb1)))
      return cosine_sim

### Path Of Main Uploaded Image

In [24]:
img1_path = "C:\\Users\\DELL\\Downloads\\shortshirt.jpg"
image1 = cv2.imread(img1_path, cv2.IMREAD_ANYCOLOR)  
cv2.imshow("clothes",image1)
cv2.waitKey(0)

-1

### Getting feature vector of the main uploaded image 

In [25]:
helper = TensorVector(img1_path)
vector1 = helper.process()
len(vector1)

1280

### Looping through images in the dataset and comparing each image feature vector with the main image feature vector

In [ ]:
for x in range(22,24):
  zeros_num = 6-len(str(x))
  zeros = ""
  for k in range(zeros_num):
    zeros+="0"
  img2_path = "E:/Graduation Project/Datasets/Deep Fashion/train/image/"+zeros+str(x)+".jpg"
  helper2 = TensorVector(img2_path)
  vector2 = helper2.process()

  print(img2_path)
  image = cv2.imread(img2_path, cv2.IMREAD_ANYCOLOR)  
  cosine_similarity = Similarity(vector1, vector2).cosineSimilarity()
  print(cosine_similarity)

  if cosine_similarity > 0.80:
    print("Cosine Similarity of The Main Image and Image:"+ img2_path + " is: "+ str(round(cosine_similarity*100))+"%")
    cv2.imshow("Clothes",image)
    cv2.waitKey(0)